In [1]:
import spacy

In [2]:
nlp = spacy.load("en_core_web_sm")
len(nlp.vocab.strings)

1139

In [3]:
coffe_hash = nlp.vocab.strings['coffee']
print(coffe_hash)

3197928453018144401


In [4]:
for i in range(10):
    print(nlp.vocab.strings[i])


IS_ALPHA
IS_ASCII
IS_DIGIT
IS_LOWER
IS_PUNCT
IS_SPACE
IS_TITLE
IS_UPPER
LIKE_URL


In [5]:
doc = nlp("I love coffee")
print('hash value:', nlp.vocab.strings['coffee'])
print('string value:', nlp.vocab.strings[3197928453018144401])

hash value: 3197928453018144401
string value: coffee
